<a href="https://colab.research.google.com/github/PatricRc/Finance_Dashboard/blob/main/Dashboard_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q

     |████████████████████████████████| 9.1 MB 4.8 MB/s 
     |████████████████████████████████| 76 kB 2.9 MB/s 
     |████████████████████████████████| 178 kB 28.0 MB/s 
     |████████████████████████████████| 180 kB 10.3 MB/s 
     |████████████████████████████████| 111 kB 41.7 MB/s 
     |████████████████████████████████| 4.3 MB 12.7 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 126 kB 59.3 MB/s 
     |████████████████████████████████| 791 kB 41.2 MB/s 
     |████████████████████████████████| 374 kB 31.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.23 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.6.0 which is incompatible.
google-colab 1.0.0 requires ipytho

In [ ]:
!pip install yfinance -q

     |████████████████████████████████| 6.3 MB 7.3 MB/s 


In [ ]:
!pip install pycoingecko -q

In [ ]:
%%writefile app.py
import pandas_datareader.data as reader
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import numpy as np
import streamlit as st
from keras.models import load_model
import pandas as pd
import requests
import tweepy
import config #archivo creado con los tokens de acceso a twitter
from pycoingecko import CoinGeckoAPI
from datetime import datetime, timedelta
cg = CoinGeckoAPI()


dashboards = st.sidebar.selectbox('Escoge un dashboard', ('Analisis financiero', 'Graficas', 'Tweepy', 'Calculadora', 'Patrones', 'Modelo de predicción'))
st.title(dashboards)

auth = tweepy.OAuthHandler(config.TWITTER_CONSUMER_KEY, config.TWITTER_CONSUMER_SECRET)
auth.set_access_token(config.TWITTER_ACCESS_TOKEN, config.TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


if dashboards == 'Analisis financiero':
    st.subheader('Este es el dashboard con analisis financieros')
    user_input = st.text_input('Selecciona tu activo', 'TSLA')

    start = st.date_input('Fecha inicial', value = pd.to_datetime('2021-01-01'))
    end = st.date_input('Fecha final', value = pd.to_datetime('today'))
      
    st.header('Tabla de datos de {}'.format(user_input))
    df2 = yf.download(user_input,start,end).dropna()
    st.dataframe(df2.tail(50))
      
    def relativeret(df):
        rel = df.pct_change()
        cumret = (1+rel).cumprod()-1
        cumret = cumret.fillna(0)
        return cumret

    df = relativeret(yf.download(user_input, start, end)['Adj Close'])
    
    st.header('Analisis descriptivo de {}'.format(user_input))
    st.write(df.describe())

if dashboards == 'Graficas':
    st.header('Este es el dashboard con las graficas')
    user_input = st.text_input('Selecciona tu activo', 'TSLA')
    start = st.date_input('Fecha inicial', value = pd.to_datetime('2021-01-01'))
    end = st.date_input('Fecha final', value = pd.to_datetime('today'))
    
    def relativeret(df):
      rel = df.pct_change()
      cumret = (1+rel).cumprod()-1
      cumret = cumret.fillna(0)
      return cumret
    
    df = relativeret(yf.download(user_input, start, end)['Adj Close'])
    df2 = yf.download(user_input,start,end).dropna()

    st.subheader('Retorno acumulado de {}'.format(user_input))
    st.line_chart(df)
  
    st.subheader('Precio de cierre vs Tiempo de {}'.format(user_input))
    fig = plt.figure(figsize= (12,6))
    plt.plot(df2['Adj Close'])
    st.pyplot(fig)

    st.subheader('Precio de cierre vs Tiempo con 100MA de {}'.format(user_input))
    ma100 = df2['Adj Close'].rolling(100).mean()
    fig = plt.figure(figsize= (12,6))
    plt.plot(ma100)
    plt.plot(df2['Adj Close'])
    st.pyplot(fig)

    st.subheader('Precio de cierre vs Tiempo con 200MA de {}'.format(user_input))
    ma100 = df2['Adj Close'].rolling(100).mean()
    ma200 = df2['Adj Close'].rolling(200).mean()
    fig = plt.figure(figsize= (12,6))
    plt.plot(ma100)
    plt.plot(ma200)
    plt.plot(df2['Adj Close'])
    st.pyplot(fig)


if dashboards == 'Tweepy':
    st.subheader('Este es el dashboard con analisis de principales tweets')
    tweets = api.user_timeline('elonmusk')
    st.write(tweets.text)

if dashboards == 'Modelo de predicción':
    st.subheader('Predicción de precio usando modelo de ML')
    user_input = st.text_input('Selecciona tu activo', 'TSLA')

    start = st.date_input('Fecha inicial', value = pd.to_datetime('2021-01-01'))
    end = st.date_input('Fecha final', value = pd.to_datetime('today'))
    
    def relativeret(df):
      rel = df.pct_change()
      cumret = (1+rel).cumprod()-1
      cumret = cumret.fillna(0)
      return cumret
    
    df = relativeret(yf.download(user_input, start, end)['Adj Close'])
    df2 = yf.download(user_input,start,end).dropna()
    df_entrenado = pd.DataFrame(df2['Adj Close'][0:int(len(df)*0.70)])
    df_prueba = pd.DataFrame(df2['Adj Close'][int(len(df)*0.70) : int(len(df))])

    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0,1))

    df_entrenado_array = scaler.fit_transform(df_entrenado)
# cargar modelo de ML
    model = load_model('modelo_keras.h5')

# Prueba del modelo de ML
    pasados_100d = df_entrenado.tail(100)
    df_final = pasados_100d.append(df_prueba, ignore_index=True)
    input_df = scaler.fit_transform(df_final)

    x_prueba = []
    y_prueba = []

    for i in range(100,input_df.shape[0]):
        x_prueba.append(input_df[i-100: i])
        y_prueba.append(input_df[i,0])

    x_prueba, y_prueba = np.array(x_prueba), np.array(y_prueba)
    y_predicted = model.predict(x_prueba)
    scaler = scaler.scale_

    scale_factor = 1/scaler[0]
    y_predicted = y_predicted * scale_factor
    y_prueba = y_prueba * scale_factor

    st.subheader('Data Precio Prediccion')
    st.dataframe(y_predicted)

# Grafico del modelo ML
    st.subheader('Precio Prediccion vs Original')
    fig2 = plt.figure(figsize=(12,6))
    plt.plot(y_prueba, 'b', label='Precio original')
    plt.plot(y_predicted, 'g', label='Precio pronosticado')
    plt.xlabel('Año')
    plt.ylabel('Precio')
    plt.legend()
    st.pyplot(fig2)

if dashboards == 'Calculadora':
    st.subheader('Esta es la calculadora de ahorro crypto')
    st.write("""
    # Bitcoin FOMO Calculator
    Use the option below to set your parameters for:
    Usa las opciones para elegir los parametros:
    - Fecha en la que quisieras haber comprado $BTC
    - Cantidad en $ que hubieras invertido 
    """)
    st.write('---')
    
    st.image('btc-nft.jpeg', use_column_width=True)
    
    #Cargar datos
    btc_current = cg.get_price(ids = 'bitcoin', vs_currencies='usd')['bitcoin']['usd']    
    #Giving Choices
    st.write('''# Escoge fecha y monto $''')
    today = datetime.utcnow().date()
    previous_day = today - timedelta(days=1)
    HIST_DATE = st.date_input("Fecha: ", value=previous_day, min_value=datetime(2014,1,1), max_value=previous_day)
    ORG_USD = st.number_input("Cantidad en $: ", min_value=1, max_value=999999999)

    #Formateando datos de fecha
    HIST_DATE_REFORMAT = HIST_DATE.strftime("%d-%m-%Y")
    HIST_DATE_datetime = datetime.strptime(HIST_DATE_REFORMAT,"%d-%m-%Y")
    btc_historic = cg.get_coin_history_by_id(id='bitcoin', vs_currencies='usd', date=HIST_DATE_REFORMAT)['market_data']['current_price']['usd']

    btc_historic = round(btc_historic, 5)

    st.write('''# Results''')
    st.write('''## Historic Analysis''')
    st.write("You would have originally bought: ***{:,.2f}*** $BTC".format(round((ORG_USD/btc_historic),5)))
    st.write("At a price of ***{:,.9f}*** per $BTC".format(btc_historic))
    st.write(" ")

    st.write('''## Present Effects''')
    total_btc = ORG_USD/btc_historic
    current_USD = total_btc * btc_current
    perc_change = (current_USD - ORG_USD)/(ORG_USD)*100
    usd_diff = current_USD - ORG_USD


Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://104.196.184.235:8501

npx: installed 22 in 4.985s
your url is: https://weak-yak-27.loca.lt
2021-12-01 16:44:39.340 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 354, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 12, in <module>
    import config #archivo creado con los tokens de acceso a twitter
ModuleNotFoundError: No module named 'config'

